In [1]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_mistralai import ChatMistralAI
from langchain_huggingface import HuggingFaceEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

import os
import getpass

In [2]:
client = QdrantClient(
    path='../db/qdrant_db',
)

collection_name='books-rec-project3'

/tmp/ipykernel_54684/4078310450.py:1: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <books-rec-project> contains 67582 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client = QdrantClient(
/tmp/ipykernel_54684/4078310450.py:1: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <books-rec-project3> contains 67553 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client = QdrantClient(


In [3]:
COLLECTION_NAME = 'books-rec-project3'

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vector_store = QdrantVectorStore(
        client=client,
        collection_name=COLLECTION_NAME,
        embedding=hf
    )

/home/o13g/bootcamp/ds-phase-2/rec-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
API_TOKEN = 'eTH2MRqGGG9TfSe1tyUR3LVUnksHx9fA'

llm = ChatMistralAI(
    model = 'mistral-small',
    temperature = .7,
    max_tokens=2000,
    api_key=API_TOKEN
)


In [5]:
# answer = llm.invoke(messages).content
# print(answer)

In [6]:
from langchain.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты эксперт-аналитик книгам с многолетним опытом и отличным чувством юмора! 🎯
    Твоя задача - проанализировать предоставленные аннотации к книгам и предугадать сюжет.

    Стиль анализа:
    - Структурируй ответ с эмодзи и забавными комментариями
    - Отвечай на русском языке живым тоном
    - Generate your response in russian language

    Помни: юмор должен быть добрым и не оскорбительным. Цель - сделать анализ интересным!

    Если среди аннотаций есть что-то особенно забавное - обязательно это отметь! 😄"""),

    ("human", """📊 ДАННЫЕ ДЛЯ ЭКСПЕРТНОГО МНЕНИЯ:
{context}

🎯 ЗАПРОС НА ЭКСПЕРТИЗУ: {question}""")
])

In [7]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [8]:
def format_docs(docs):
    """Форматирует документы для передачи в промпт"""
    formatted = []

    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata

        vacancy_info = f"""
        === Книга {i} ===
        Авторы: {metadata.get('author', 'Не указано')}
        Название: {metadata.get('title', 'Не указано')}
        Картинка: {metadata.get('img_url', 'Не указано')}
        Сылка: {metadata.get('book_url', 'Не указано')}

        Краткое Описание: {metadata.get('annotation', 'Не указано')}...
        """

        formatted.append(vacancy_info)

    return "\n".join(formatted)

print("✅ Функция форматирования создана")


✅ Функция форматирования создана


In [9]:
# Создаем RAG цепочку
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    } # словарь, в котором ключи - это переменные, которые будут переданы в промпт
    | rag_prompt # промпт для RAG
    | llm # тут можно поставить любую llm-модель
    | StrOutputParser() # для вывода ответа в читаемом виде
)

print("✅ RAG цепочка создана")

✅ RAG цепочка создана


In [10]:
# Тестируем
question = "Что ты думаешь об этой Книги, ее Название и авторе?"
try:
    answer = rag_chain.invoke(question)
    print("🔍 ОТВЕТ RAG:")
    print(answer)
except Exception as e:
    print(f"❌ Ошибка: {e}")

🔍 ОТВЕТ RAG:
📚 Ох, книги! Вот сколько интересного и загадочного в этом списке! 😄

**📕 Книга 1 - "Реминисценция" Дмитрия Рощина**
🔎 Судя по названию, здесь можно ожидать чего-то вроде воспоминаний или отголосков прошлого. Сюрреалистическая новелла - это звучит увлекательно! Тут, наверное, будут странные связи, неожиданные повороты и завораживающие метафоры. 🎩 Хочется надеть феску детектива и разгадывать загадки вместе с автором!

**📕 Книга 2 - "Глаза из прошлого" Домоседа**
🙆‍♂️ Первый творческий опыт автора! Это всегда здорово! История, вероятно, будет набирать обороты по мере развития навыков писателя. Может, там спрятаны неожиданные перлы? Я рад буду любой адекватной критике, - говорит Домосед. Ну, я готов стать его первым читателем и подружиться с его уникальным стилем! 🤝

**📕 Книга 3 - "История до... Пантеон" Василия Барда**
🤔 К сожалению, аннотация неразборчива, но я готов рискнуть! Возможно, там будет история о путешествии, возрождении или открытии чего-то нового. Пусть нас занес

In [11]:
def format_book(book):
    formatted_book = f"""
    Авторы: {book.get('author', 'Не указано')}
    Название: {book.get('title', 'Не указано')}
    Картинка: {book.get('img_url', 'Не указано')}
    Сылка: {book.get('book_url', 'Не указано')}

    Краткое Описание: {book.get('annotation', 'Не указано')}...
    """

    return formatted_book

In [ ]:
def demonstrate_rag_step_by_step(book:dict):
    query = 'Что ты думаешь про эту книгу, ее название и ее автора?'
    formatted_context = format_book(book)
    input_data = {
        "context": formatted_context,
        "question": query
    }
    formatted_prompt = rag_prompt.format(**input_data)
    
    # Получаем ответ
    response = llm.invoke(formatted_prompt)
    final_answer = StrOutputParser().parse(response)

    return final_answer.content

In [13]:
book = {'author': 'Aru Alimbet', 'title': 'Ночная сделка', 'main_genre': 'Фэнтези', 'annotation': 'Она живёт среди цифр, где каждая транзакция — просто движение капитала, а каждый отчёт — всего лишь работа. До тех пор, пока одна строка в базе данных не начинае...', 'img_url': 'https://rust.litnet.com/uploads/covers/120/1747997045_38.jpg', 'book_url': 'https://litnet.com/ru/book/xolodnym-utrom-b532882', '_id': '73430f03-facc-46af-8922-7eb25753fcbe', '_collection_name': 'books-rec-project3'}
answer = demonstrate_rag_step_by_step(book)

In [ ]:
answer.content

AIMessage(content='Анализ🧐:\n\nВ книге "Ночная сделка" автора Ару Альimbет, кажется, что главная героиня будет кого-то LOL! "Она живёт среди цифр" - это уже намекает на то, что наша героиня связана с математикой, анализом данных или финансами. 💰\n\nНо ожидайте неожиданности, потому что "одна строка в базе данных неожиданно начинает..." - это как будто намекает на какой-то сюрприз или интригу в среде цифр и транзакций! 💡\n\nАвтор, Ару Альimbет, явно имеет хороший юмор, выбрав "Ночная сделка" в качестве названия. Это звучит таинственно и забавно одновременно! 😂\n\nВ целом, я думаю, что эта книга будет увлекательной и забавной, с неожиданными поворотами и забавными ситуациями в мире финансов. Жду продолжения! 👀\n\nПосмотрите, пожалуйста, на обложку и ссылку, которые были предоставлены, чтобы узнать больше! 👉 [https://litnet.com/ru/book/xolodnym-utrom-b532882](https://litnet.com/ru/book/xolodnym-utrom-b532882) 🌐', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 49

In [15]:
client.close()